In [7]:
import pandas as pd
import urllib.request
import os.path
from zipfile import ZipFile
from simpledbf import Dbf5

In [13]:
#Import the data files
#Create a dictionary of the urls and the corresponding zipnames
data_dict={}
data_dict['chem'] = {'url': "https://www.waterboards.ca.gov/drinking_water/certlic/drinkingwater/documents/edtlibrary/chemical.zip",
                    'filename':"chem_import.zip",'filetype':'zip'}
data_dict['storet'] = {'url': "https://www.waterboards.ca.gov/drinking_water/certlic/drinkingwater/documents/edtlibrary/storet.zip",
                      'filename':"storet_import.zip",'filetype':'zip'}
data_dict['geo'] = {'url':"https://data.ca.gov/sites/default/files/Public%20Potable%20Water%20Systems%20FINAL%2006-22-2018_0.csv",
                   'filename':'geo_import.csv','filetype':'csv'}

for k in data_dict.keys():
    dat_temp=data_dict[k]
    if (os.path.exists(dat_temp['filename'])==False):
        print('Retrieving file and saving as {:}'.format(dat_temp['filename']))
        urllib.request.urlretrieve(dat_temp['url'],dat_temp['filename'])


Retrieving file and saving as geo_import.csv


In [ ]:
#Unzip the files and store the contents
data_store = {}
for k in data_dict.keys():
        dat_temp=data_dict[k]
        if dat_temp['filetype']=='zip':
            with ZipFile(dat_temp['filename'], 'r') as zipObj:
                fname=zipObj.namelist()
                zipObj.extractall()
                #Open the database file
                dbf = Dbf5(fname[0])
                data_store[k] = dbf.to_dataframe()
        else:
            data_store[k] = pd.read_csv(dat_temp['filename'])
            

In [ ]:
#Clean up some of the data

